In [7]:
pip install tkinter

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tkcalendar


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Part 1: Required imports
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
from datetime import datetime
import sqlite3

In [3]:
# Part 2: Database Setup
def setup_database():
    conn = sqlite3.connect('attendancewage_system.db')
    cursor = conn.cursor()
    
    # Create Employee table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Employee (
            employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            designation TEXT NOT NULL,
            base_salary REAL NOT NULL,
            annual_increment BOOLEAN DEFAULT TRUE,
            joining_date TEXT,
            address TEXT,
            aadhar_card TEXT,
            voter_card TEXT,
            pan_no TEXT,
            mobile_no TEXT,
            emergency_contact TEXT,
            blood_group TEXT,
            is_part_time BOOLEAN DEFAULT FALSE
        )
    ''')
        
    # Create MonthlyAttendance table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MonthlyAttendance (
            employee_id INTEGER,
            date TEXT,
            status TEXT,
            holiday_count INTEGER DEFAULT 0,
            FOREIGN KEY (employee_id) REFERENCES Employee (employee_id)
        )
    ''')
    
    # Create HolidayTracking table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS HolidayTracking (
            employee_id INTEGER,
            month TEXT,
            holidays_taken INTEGER DEFAULT 0,
            FOREIGN KEY (employee_id) REFERENCES Employee (employee_id)
        )
    ''')
    
    conn.commit()
    conn.close()


In [8]:
# Part 3: Main Application Class
class EmployeeAttendanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Employee Attendance and Wage System")
        self.root.geometry("800x600")
        
        # Database connection
        self.conn = sqlite3.connect('attendancewage_system.db')
        self.cursor = self.conn.cursor()
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill="both", padx=10, pady=5)
        
        # Initialize frames
        self.employee_frame = ttk.Frame(self.notebook)
        self.attendance_frame = ttk.Frame(self.notebook)
        self.salary_frame = ttk.Frame(self.notebook)
        
        # Add frames to notebook
        self.notebook.add(self.employee_frame, text="Employee Management")
        self.notebook.add(self.attendance_frame, text="Attendance Management")
        self.notebook.add(self.salary_frame, text="Salary Management")
        
        # Setup individual tabs
        self.setup_employee_tab()
        self.setup_attendance_tab()
        self.setup_salary_tab()
    
    def setup_employee_tab(self):
        # Create a frame for the form
        form_frame = ttk.LabelFrame(self.employee_frame, text="Employee Details")
        form_frame.pack(fill="x", padx=10, pady=5)
        
        # Employee form fields
        fields = [
            ("Name:", "name_entry"),
            ("Designation:", "designation_entry"),
            ("Base Salary:", "salary_entry"),
            ("Address:", "address_entry"),
            ("Aadhar Card:", "aadhar_entry"),
            ("Voter Card:", "voter_entry"),
            ("PAN No:", "pan_entry"),
            ("Mobile No:", "mobile_entry"),
            ("Emergency Contact:", "emergency_entry"),
            ("Blood Group:", "blood_group_entry")
        ]
        
        for i, (label_text, entry_name) in enumerate(fields):
            ttk.Label(form_frame, text=label_text).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(form_frame)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="ew")
            setattr(self, entry_name, entry)
        
        # Part-time checkbox
        self.is_part_time_var = tk.BooleanVar()
        ttk.Checkbutton(
            form_frame,
            text="Part-Time Employee",
            variable=self.is_part_time_var
        ).grid(row=len(fields), column=0, columnspan=2, pady=5)
        
        # Buttons
        button_frame = ttk.Frame(form_frame)
        button_frame.grid(row=len(fields)+1, column=0, columnspan=2, pady=10)
        
        ttk.Button(button_frame, text="Add Employee", command=self.add_employee).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Clear Form", command=self.clear_form).pack(side=tk.LEFT, padx=5)
        
        # Employee List
        list_frame = ttk.LabelFrame(self.employee_frame, text="Employee List")
        list_frame.pack(fill="both", expand=True, padx=10, pady=5)
        
        self.tree = ttk.Treeview(
            list_frame,
            columns=("ID", "Name", "Designation", "Salary"),
            show="headings"
        )
        
        # Configure columns
        for col in ("ID", "Name", "Designation", "Salary"):
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        
        self.tree.pack(fill="both", expand=True)
        self.refresh_employee_list()
    
    def setup_attendance_tab(self):
        # Employee selection
        select_frame = ttk.LabelFrame(self.attendance_frame, text="Record Attendance")
        select_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(select_frame, text="Select Employee:").grid(row=0, column=0, padx=5, pady=5)
        self.attendance_employee_var = tk.StringVar()
        self.employee_dropdown = ttk.Combobox(
            select_frame,
            textvariable=self.attendance_employee_var
        )
        self.employee_dropdown.grid(row=0, column=1, padx=5, pady=5)
        
        # Date selection
        ttk.Label(select_frame, text="Date:").grid(row=1, column=0, padx=5, pady=5)
        self.attendance_date = DateEntry(select_frame, width=12)
        self.attendance_date.grid(row=1, column=1, padx=5, pady=5)
        
        # Status selection
        ttk.Label(select_frame, text="Status:").grid(row=2, column=0, padx=5, pady=5)
        self.status_var = tk.StringVar()
        status_frame = ttk.Frame(select_frame)
        status_frame.grid(row=2, column=1, padx=5, pady=5)
        
        statuses = ["Present", "Half Day", "Absent"]
        for i, status in enumerate(statuses):
            ttk.Radiobutton(
                status_frame,
                text=status,
                variable=self.status_var,
                value=status
            ).pack(side=tk.LEFT, padx=5)
        
        # Record button
        ttk.Button(
            select_frame,
            text="Record Attendance",
            command=self.record_attendance
        ).grid(row=3, column=0, columnspan=2, pady=10)
        
        # Attendance List
        list_frame = ttk.LabelFrame(self.attendance_frame, text="Attendance Records")
        list_frame.pack(fill="both", expand=True, padx=10, pady=5)
        
        self.attendance_tree = ttk.Treeview(
            list_frame,
            columns=("Date", "Employee", "Status"),
            show="headings"
        )
        
        for col in ("Date", "Employee", "Status"):
            self.attendance_tree.heading(col, text=col)
            self.attendance_tree.column(col, width=100)
        
        self.attendance_tree.pack(fill="both", expand=True)
        self.refresh_attendance_list()
        
    def setup_salary_tab(self):
        # Date range selection
        date_frame = ttk.LabelFrame(self.salary_frame, text="Select Period")
        date_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(date_frame, text="From:").grid(row=0, column=0, padx=5, pady=5)
        self.salary_from_date = DateEntry(date_frame, width=12)
        self.salary_from_date.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(date_frame, text="To:").grid(row=0, column=2, padx=5, pady=5)
        self.salary_to_date = DateEntry(date_frame, width=12)
        self.salary_to_date.grid(row=0, column=3, padx=5, pady=5)
        
        # Employee selection
        select_frame = ttk.Frame(self.salary_frame)
        select_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(select_frame, text="Select Employee:").pack(side=tk.LEFT, padx=5)
        self.salary_employee_var = tk.StringVar()
        self.salary_employee_dropdown = ttk.Combobox(
            select_frame,
            textvariable=self.salary_employee_var
        )
        self.salary_employee_dropdown.pack(side=tk.LEFT, padx=5)
        
        # Calculate button
        ttk.Button(
            self.salary_frame,
            text="Calculate Salary",
            command=self.calculate_salary
        ).pack(pady=10)
        
        # Salary details display
        self.salary_text = tk.Text(self.salary_frame, height=10, width=50)
        self.salary_text.pack(padx=10, pady=5, fill="both", expand=True)
    
    # Database operations
    def add_employee(self):
        try:
            self.cursor.execute('''
                INSERT INTO Employee (
                    name, designation, base_salary, address, aadhar_card,
                    voter_card, pan_no, mobile_no, emergency_contact,
                    blood_group, is_part_time
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                self.name_entry.get(),
                self.designation_entry.get(),
                float(self.salary_entry.get()),
                self.address_entry.get(),
                self.aadhar_entry.get(),
                self.voter_entry.get(),
                self.pan_entry.get(),
                self.mobile_entry.get(),
                self.emergency_entry.get(),
                self.blood_group_entry.get(),
                self.is_part_time_var.get()
            ))
            self.conn.commit()
            messagebox.showinfo("Success", "Employee added successfully!")
            self.clear_form()
            self.refresh_employee_list()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add employee: {str(e)}")
    
    def record_attendance(self):
        try:
            employee_name = self.attendance_employee_var.get()
            self.cursor.execute('SELECT employee_id FROM Employee WHERE name = ?', (employee_name,))
            employee_id = self.cursor.fetchone()[0]
            
            self.cursor.execute('''
                INSERT INTO MonthlyAttendance (employee_id, date, status)
                VALUES (?, ?, ?)
            ''', (
                employee_id,
                self.attendance_date.get_date().strftime('%Y-%m-%d'),
                self.status_var.get()
            ))
            self.conn.commit()
            messagebox.showinfo("Success", "Attendance recorded successfully!")
            self.refresh_attendance_list()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to record attendance: {str(e)}")
    
    def calculate_salary(self):
        try:
            employee_name = self.salary_employee_var.get()
            from_date = self.salary_from_date.get_date()
            to_date = self.salary_to_date.get_date()
            
            self.cursor.execute('''
                SELECT employee_id, base_salary, is_part_time
                FROM Employee
                WHERE name = ?
            ''', (employee_name,))
            employee_data = self.cursor.fetchone()
            
            if not employee_data:
                raise ValueError("Employee not found")
                
            employee_id, base_salary, is_part_time = employee_data
            
            # Calculate working days
            self.cursor.execute('''
                SELECT COUNT(*) as present_days,
                       SUM(CASE WHEN status = 'Half Day' THEN 0.5 ELSE 1 END) as effective_days
                FROM MonthlyAttendance
                WHERE employee_id = ? AND date BETWEEN ? AND ? AND status != 'Absent'
            ''', (employee_id, from_date, to_date))
            
            attendance_data = self.cursor.fetchone()
            present_days = attendance_data[0] or 0
            effective_days = attendance_data[1] or 0
            
            # Calculate salary
            daily_rate = base_salary / 30
            calculated_salary = daily_rate * effective_days
            
            # Display results
            self.salary_text.delete(1.0, tk.END)
            self.salary_text.insert(tk.END, f"""
Salary Calculation for {employee_name}
Period: {from_date.strftime('%Y-%m-%d')} to {to_date.strftime('%Y-%m-%d')}

Base Salary: ₹{base_salary:.2f}
Present Days: {present_days}
Effective Working Days: {effective_days}
Daily Rate: ₹{daily_rate:.2f}

Calculated Salary: ₹{calculated_salary:.2f}
            """)
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to calculate salary: {str(e)}")
    
    # Utility methods
    def refresh_employee_list(self):
        for item in self.tree.get_children():
            self.tree.delete(item)
            
        self.cursor.execute('SELECT employee_id, name, designation, base_salary FROM Employee')
        for row in self.cursor.fetchall():
            self.tree.insert('', tk.END, values=row)
            
        # Update employee dropdowns
        employee_list = [row[1] for row in self.cursor.execute('SELECT employee_id, name FROM Employee')]
        self.employee_dropdown['values'] = employee_list
        self.salary_employee_dropdown['values'] = employee_list
    
    def refresh_attendance_list(self):
        for item in self.attendance_tree.get_children():
            self.attendance_tree.delete(item)
            
        self.cursor.execute('''
            SELECT a.date, e.name, a.status
            FROM MonthlyAttendance a
            JOIN Employee e ON a.employee_id = e.employee_id
            ORDER BY a.date DESC
        ''')
        for row in self.cursor.fetchall():
            self.attendance_tree.insert('', tk.END, values=row)
    
    def clear_form(self):
        """Clear all entry fields in the employee form."""
        for attr in dir(self):
            if attr.endswith('_entry'):
                getattr(self, attr).delete(0, tk.END)
        self.is_part_time_var.set(False)
    
    def __del__(self):
        """Destructor to ensure database connection is closed."""
        if hasattr(self, 'conn') and self.conn:
            self.conn.close()

# Main execution
if __name__ == "__main__":
    # Setup the database first
    setup_database()
    
    # Create and run the application
    root = tk.Tk()
    app = EmployeeAttendanceApp(root)
    root.mainloop()

AttributeError: 'EmployeeAttendanceApp' object has no attribute 'employee_dropdown'

In [3]:
# Part 1: Required imports
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
from datetime import datetime
import sqlite3

# Part 2: Database Setup
def setup_database():
    conn = sqlite3.connect('attendancewage_system.db')
    cursor = conn.cursor()
    
    # Create Employee table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Employee (
            employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            designation TEXT NOT NULL,
            base_salary REAL NOT NULL,
            annual_increment BOOLEAN DEFAULT TRUE,
            joining_date TEXT,
            address TEXT,
            aadhar_card TEXT,
            voter_card TEXT,
            pan_no TEXT,
            mobile_no TEXT,
            emergency_contact TEXT,
            blood_group TEXT,
            is_part_time BOOLEAN DEFAULT FALSE
        )
    ''')
    
    # Create MonthlyAttendance table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MonthlyAttendance (
            employee_id INTEGER,
            date TEXT,
            status TEXT,
            holiday_count INTEGER DEFAULT 0,
            FOREIGN KEY (employee_id) REFERENCES Employee (employee_id)
        )
    ''')
    
    conn.commit()
    conn.close()

class EmployeeAttendanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Employee Attendance and Wage System")
        self.root.geometry("800x600")
        
        # Database connection
        self.conn = sqlite3.connect('attendancewage_system.db')
        self.cursor = self.conn.cursor()
        
        # Initialize employee list
        self.employee_list = []
        self.update_employee_list()
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill="both", padx=10, pady=5)
        
        # Initialize frames
        self.employee_frame = ttk.Frame(self.notebook)
        self.attendance_frame = ttk.Frame(self.notebook)
        self.salary_frame = ttk.Frame(self.notebook)
        
        # Add frames to notebook
        self.notebook.add(self.employee_frame, text="Employee Management")
        self.notebook.add(self.attendance_frame, text="Attendance Management")
        self.notebook.add(self.salary_frame, text="Salary Management")
        
        # Setup individual tabs
        self.setup_employee_tab()
        self.setup_attendance_tab()
        self.setup_salary_tab()

    def update_employee_list(self):
        """Update the list of employees from database"""
        self.cursor.execute('SELECT employee_id, name FROM Employee ORDER BY name')
        self.employee_list = [(row[0], row[1]) for row in self.cursor.fetchall()]
        
    def setup_attendance_tab(self):
        # Employee selection
        select_frame = ttk.LabelFrame(self.attendance_frame, text="Record Attendance")
        select_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Label(select_frame, text="Select Employee:").grid(row=0, column=0, padx=5, pady=5)
        
        # Initialize the employee dropdown with a StringVar
        self.attendance_employee_var = tk.StringVar()
        self.employee_dropdown = ttk.Combobox(
            select_frame,
            textvariable=self.attendance_employee_var,
            values=[emp[1] for emp in self.employee_list]  # Use employee names only
        )
        self.employee_dropdown.grid(row=0, column=1, padx=5, pady=5)
        
        # Date selection
        ttk.Label(select_frame, text="Date:").grid(row=1, column=0, padx=5, pady=5)
        self.attendance_date = DateEntry(select_frame, width=12)
        self.attendance_date.grid(row=1, column=1, padx=5, pady=5)
        
        # Status selection
        ttk.Label(select_frame, text="Status:").grid(row=2, column=0, padx=5, pady=5)
        self.status_var = tk.StringVar(value="Present")  # Set default value
        status_frame = ttk.Frame(select_frame)
        status_frame.grid(row=2, column=1, padx=5, pady=5)
        
        statuses = ["Present", "Half Day", "Absent"]
        for i, status in enumerate(statuses):
            ttk.Radiobutton(
                status_frame,
                text=status,
                variable=self.status_var,
                value=status
            ).pack(side=tk.LEFT, padx=5)
        
        # Record button
        ttk.Button(
            select_frame,
            text="Record Attendance",
            command=self.record_attendance
        ).grid(row=3, column=0, columnspan=2, pady=10)
        
        # Attendance List
        list_frame = ttk.LabelFrame(self.attendance_frame, text="Attendance Records")
        list_frame.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Create and configure the Treeview
        self.attendance_tree = ttk.Treeview(
            list_frame,
            columns=("Date", "Employee", "Status"),
            show="headings"
        )
        
        # Configure scrollbar
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=self.attendance_tree.yview)
        scrollbar.pack(side="right", fill="y")
        
        self.attendance_tree.configure(yscrollcommand=scrollbar.set)
        
        # Configure columns
        for col in ("Date", "Employee", "Status"):
            self.attendance_tree.heading(col, text=col)
            self.attendance_tree.column(col, width=100)
        
        self.attendance_tree.pack(fill="both", expand=True)
        
        # Initial refresh of attendance list
        self.refresh_attendance_list()

    def refresh_attendance_list(self):
        """Refresh the attendance list and employee dropdowns"""
        # Clear existing items in the attendance tree
        for item in self.attendance_tree.get_children():
            self.attendance_tree.delete(item)
            
        # Update employee list
        self.update_employee_list()
        
        # Update attendance records
        self.cursor.execute('''
            SELECT a.date, e.name, a.status
            FROM MonthlyAttendance a
            JOIN Employee e ON a.employee_id = e.employee_id
            ORDER BY a.date DESC
        ''')
        
        for row in self.cursor.fetchall():
            self.attendance_tree.insert('', tk.END, values=row)
        
        # Update employee dropdowns
        employee_names = [emp[1] for emp in self.employee_list]
        if hasattr(self, 'employee_dropdown'):
            self.employee_dropdown['values'] = employee_names
        if hasattr(self, 'salary_employee_dropdown'):
            self.salary_employee_dropdown['values'] = employee_names

    def record_attendance(self):
        """Record attendance for selected employee"""
        employee_name = self.attendance_employee_var.get()
        if not employee_name:
            messagebox.showerror("Error", "Please select an employee")
            return
            
        try:
            # Get employee_id for selected name
            employee_id = None
            for emp_id, emp_name in self.employee_list:
                if emp_name == employee_name:
                    employee_id = emp_id
                    break
            
            if employee_id is None:
                raise ValueError("Employee not found")
                
            # Insert attendance record
            self.cursor.execute('''
                INSERT INTO MonthlyAttendance (employee_id, date, status)
                VALUES (?, ?, ?)
            ''', (
                employee_id,
                self.attendance_date.get_date().strftime('%Y-%m-%d'),
                self.status_var.get()
            ))
            
            self.conn.commit()
            messagebox.showinfo("Success", "Attendance recorded successfully!")
            self.refresh_attendance_list()
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to record attendance: {str(e)}")
            self.conn.rollback()

    def __del__(self):
        """Destructor to ensure database connection is closed"""
        if hasattr(self, 'conn') and self.conn:
            self.conn.close()

# Main execution
if __name__ == "__main__":
    # Setup the database first
    setup_database()
    
    # Create and run the application
    root = tk.Tk()
    app = EmployeeAttendanceApp(root)
    root.mainloop()

AttributeError: 'EmployeeAttendanceApp' object has no attribute 'setup_employee_tab'

: 

In [ ]:
pip install tkcalendar

In [ ]:
import sqlite3
import os

# Connect to SQLite database (it will create a new one since we deleted the old one)
conn = sqlite3.connect('attendancewage_system.db')
cursor = conn.cursor()

# Create Employee table with all required columns
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employee (
        employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        designation TEXT NOT NULL,
        base_salary REAL NOT NULL,
        annual_increment BOOLEAN DEFAULT TRUE,
        joining_date TEXT,
        address TEXT,
        aadhar_card TEXT,
        voter_card TEXT,
        pan_no TEXT,
        mobile_no TEXT,
        emergency_contact TEXT,
        blood_group TEXT,
        is_part_time BOOLEAN DEFAULT FALSE
    )
''')

# Create MonthlyAttendance table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS MonthlyAttendance (
        employee_id INTEGER,
        date TEXT,
        status TEXT,
        holiday_count INTEGER DEFAULT 0,
        FOREIGN KEY (employee_id) REFERENCES Employee (employee_id)
    )
''')

# Create HolidayTracking table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS HolidayTracking (
        employee_id INTEGER,
        month TEXT,
        holidays_taken INTEGER DEFAULT 0,
        FOREIGN KEY (employee_id) REFERENCES Employee (employee_id)
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database setup complete.")

Part 1: Imports and Initialization

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
from datetime import datetime
import sqlite3

Part 2: Main Application Class

In [4]:
class EmployeeAttendanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Employee Attendance System")
        
        # Notebook for tabs
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(pady=10, expand=True)
        
        # Initialize frames
        self.employee_frame = ttk.Frame(self.notebook)
        self.attendance_frame = ttk.Frame(self.notebook)
        self.salary_frame = ttk.Frame(self.notebook)
        
        # Add frames as tabs
        self.notebook.add(self.employee_frame, text="Employee Management")
        self.notebook.add(self.attendance_frame, text="Attendance Management")
        self.notebook.add(self.salary_frame, text="Salary Management")
        
        # Setup tabs
        self.setup_employee_tab()
        self.setup_attendance_tab()
        self.setup_salary_tab()


In [5]:
class EmployeeAttendanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Employee Attendance and Wage System")
        self.conn = None
        self.cursor = None
        
        self.connect_to_database()

        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill="both")
        
        # Setup tabs
        self.setup_employee_tab()
        self.setup_attendance_tab()
        self.setup_salary_tab()

Part 3: Employee Management Tab

In [21]:
def setup_employee_tab(self):
        """Set up the Employee Tab."""
        self.employee_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.employee_tab, text="Employee Management")
        
        # Employee form
        form_frame = ttk.LabelFrame(self.employee_tab, text="Add Employee")
        form_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        
        # Labels and entries
        ttk.Label(form_frame, text="Name:").grid(row=0, column=0, sticky="e")
        self.name_entry = ttk.Entry(form_frame)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(form_frame, text="Designation:").grid(row=1, column=0, sticky="e")
        self.designation_entry = ttk.Entry(form_frame)
        self.designation_entry.grid(row=1, column=1, padx=5, pady=5)

        ttk.Label(form_frame, text="Base Salary:").grid(row=2, column=0, sticky="e")
        self.salary_entry = ttk.Entry(form_frame)
        self.salary_entry.grid(row=2, column=1, padx=5, pady=5)

        ttk.Label(form_frame, text="Address:").grid(row=3, column=0, sticky="e")
        self.address_entry = ttk.Entry(form_frame)
        self.address_entry.grid(row=3, column=1, padx=5, pady=5)

        # Add more fields as needed...
        
        self.is_part_time_var = tk.BooleanVar()
        self.is_part_time_checkbox = ttk.Checkbutton(form_frame, text="Is Part-Time", variable=self.is_part_time_var)
        self.is_part_time_checkbox.grid(row=4, columnspan=2, sticky="w", padx=5)

        # Buttons
        self.add_employee_button = ttk.Button(form_frame, text="Add Employee", command=self.add_employee)
        self.add_employee_button.grid(row=5, column=0, columnspan=2, pady=10)

        # Employee list
        list_frame = ttk.LabelFrame(self.employee_tab, text="Employee List")
        list_frame.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        self.tree = ttk.Treeview(list_frame, columns=("ID", "Name", "Designation", "Salary"), show="headings")
        self.tree.heading("ID", text="ID")
        self.tree.heading("Name", text="Name")
        self.tree.heading("Designation", text="Designation")
        self.tree.heading("Salary", text="Salary")
        self.tree.pack(expand=True, fill="both")
        
        self.refresh_employee_list()


In [7]:
    def setup_employee_tab(self):
        fields = [
            ("Employee Name:", "name_entry"),
            ("Designation:", "designation_entry"),
            ("Base Salary:", "salary_entry"),
            ("Address:", "address_entry"),
            ("Aadhar Card:", "aadhar_entry"),
            ("Voter Card:", "voter_entry"),
            ("PAN No:", "pan_entry"),
            ("Mobile No:", "mobile_entry"),
            ("Emergency Contact:", "emergency_entry"),
            ("Blood Group:", "blood_group_entry")
        ]
        
        for i, (label_text, entry_name) in enumerate(fields):
            ttk.Label(self.employee_frame, text=label_text).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.employee_frame)
            entry.grid(row=i, column=1, padx=5, pady=5)
            setattr(self, entry_name, entry)
        
        # Part-time checkbox
        self.is_part_time_var = tk.BooleanVar()
        ttk.Checkbutton(
            self.employee_frame, 
            text="Part-Time Employee", 
            variable=self.is_part_time_var
        ).grid(row=len(fields), column=0, columnspan=2, pady=5)
        
        # Buttons for Employee Tab
        self.create_employee_buttons(len(fields))
        
        # Employee List
        self.tree = ttk.Treeview(self.employee_frame, columns=("ID", "Name", "Designation", "Salary"), show="headings")
        self.tree.grid(row=len(fields) + 2, column=0, columnspan=2, pady=10, padx=5)
        for col in ("ID", "Name", "Designation", "Salary"):
            self.tree.heading(col, text=col)
        
        self.refresh_employee_list()
        
    def create_employee_buttons(self, field_count):
        btn_frame = ttk.Frame(self.employee_frame)
        btn_frame.grid(row=field_count + 1, column=0, columnspan=2, pady=10)
        ttk.Button(btn_frame, text="Add Employee", command=self.add_employee).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Clear Form", command=self.clear_form).pack(side=tk.LEFT, padx=5)

Part 4: Attendance Management Tab

In [8]:
    def setup_attendance_tab(self):
        ttk.Label(self.attendance_frame, text="Select Employee:").grid(row=0, column=0, padx=5, pady=5)
        self.attendance_employee_var = tk.StringVar()
        self.employee_dropdown = ttk.Combobox(self.attendance_frame, textvariable=self.attendance_employee_var)
        self.employee_dropdown.grid(row=0, column=1, padx=5, pady=5)
        
        # Date Selection
        ttk.Label(self.attendance_frame, text="Date:").grid(row=1, column=0, padx=5, pady=5)
        self.attendance_date = DateEntry(self.attendance_frame, width=12)
        self.attendance_date.grid(row=1, column=1, padx=5, pady=5)
        
        # Status Options
        ttk.Label(self.attendance_frame, text="Status:").grid(row=2, column=0, padx=5, pady=5)
        self.status_var = tk.StringVar()
        status_frame = ttk.Frame(self.attendance_frame)
        status_frame.grid(row=2, column=1, padx=5, pady=5)
        for status in ["Present", "Half Day", "Absent"]:
            ttk.Radiobutton(status_frame, text=status, variable=self.status_var, value=status).pack(side=tk.LEFT)
        
        # Record Button
        ttk.Button(self.attendance_frame, text="Record Attendance", command=self.record_attendance).grid(row=3, column=0, columnspan=2, pady=10)
        
        # Attendance List
        self.attendance_tree = ttk.Treeview(self.attendance_frame, columns=("Date", "Employee", "Status"), show="headings")
        self.attendance_tree.grid(row=4, column=0, columnspan=2, pady=10, padx=5)
        for col in ("Date", "Employee", "Status"):
            self.attendance_tree.heading(col, text=col)
        
        self.refresh_attendance_list()

Part 5: Salary Management Tab

In [9]:
    def setup_salary_tab(self):
        # Date Range
        date_frame = ttk.LabelFrame(self.salary_frame, text="Date Range")
        date_frame.grid(row=0, column=0, columnspan=2, padx=5, pady=5)
        self.create_date_range_selector(date_frame)
        
        # Employee Dropdown
        ttk.Label(self.salary_frame, text="Select Employee:").grid(row=1, column=0, padx=5, pady=5)
        self.salary_employee_var = tk.StringVar()
        self.salary_employee_dropdown = ttk.Combobox(self.salary_frame, textvariable=self.salary_employee_var)
        self.salary_employee_dropdown.grid(row=1, column=1, padx=5, pady=5)
        
        # Calculate Button
        ttk.Button(self.salary_frame, text="Calculate Salary", command=self.calculate_salary).grid(row=2, column=0, columnspan=2, pady=10)
        
        # Salary Details
        self.salary_text = tk.Text(self.salary_frame, height=10, width=50)
        self.salary_text.grid(row=3, column=0, columnspan=2, pady=5, padx=5)

Part 6: Database Connection Setup

In [24]:
    def connect_to_database(self):
        """Connect to the SQLite database."""
        self.conn = sqlite3.connect('attendancewage_system.db')
        self.cursor = self.conn.cursor()
    
    def close_database(self):
        """Close the SQLite database connection."""
        if self.conn:
            self.conn.close()
    
    def __del__(self):
        """Destructor to ensure the database connection is closed."""
        self.close_database()

Part 7: Employee Tab Functionalities

In [11]:
    def add_employee(self):
        """Add a new employee to the database."""
        name = self.name_entry.get()
        designation = self.designation_entry.get()
        salary = self.salary_entry.get()
        address = self.address_entry.get()
        aadhar = self.aadhar_entry.get()
        voter = self.voter_entry.get()
        pan = self.pan_entry.get()
        mobile = self.mobile_entry.get()
        emergency = self.emergency_entry.get()
        blood_group = self.blood_group_entry.get()
        is_part_time = self.is_part_time_var.get()
        
        if not (name and designation and salary):
            messagebox.showerror("Error", "Name, Designation, and Base Salary are required.")
            return
        
        try:
            self.cursor.execute('''
                INSERT INTO Employee (
                    name, designation, base_salary, address, aadhar_card, voter_card,
                    pan_no, mobile_no, emergency_contact, blood_group, is_part_time
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (name, designation, salary, address, aadhar, voter, pan, mobile, emergency, blood_group, is_part_time))
            self.conn.commit()
            messagebox.showinfo("Success", "Employee added successfully!")
            self.refresh_employee_list()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to add employee. {e}")
    
    def refresh_employee_list(self):
        """Refresh the employee list in the TreeView."""
        for row in self.tree.get_children():
            self.tree.delete(row)
        
        self.cursor.execute('SELECT employee_id, name, designation, base_salary FROM Employee')
        for row in self.cursor.fetchall():
            self.tree.insert('', tk.END, values=row)
    
    def clear_form(self):
        """Clear all the input fields."""
        for attr in dir(self):
            if attr.endswith("_entry"):
                getattr(self, attr).delete(0, tk.END)
        self.is_part_time_var.set(False)

Part 8: Attendance Tab Functionalities

In [12]:
    def record_attendance(self):
        """Record attendance for a selected employee."""
        employee_name = self.attendance_employee_var.get()
        attendance_date = self.attendance_date.get_date()
        status = self.status_var.get()
        
        if not (employee_name and status):
            messagebox.showerror("Error", "Employee and Status are required.")
            return
        
        # Get employee ID
        self.cursor.execute('SELECT employee_id FROM Employee WHERE name = ?', (employee_name,))
        result = self.cursor.fetchone()
        if not result:
            messagebox.showerror("Error", "Employee not found.")
            return
        
        employee_id = result[0]
        try:
            self.cursor.execute('''
                INSERT INTO MonthlyAttendance (employee_id, date, status)
                VALUES (?, ?, ?)
            ''', (employee_id, attendance_date, status))
            self.conn.commit()
            messagebox.showinfo("Success", "Attendance recorded successfully!")
            self.refresh_attendance_list()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to record attendance. {e}")
    
    def refresh_attendance_list(self):
        """Refresh the attendance list in the TreeView."""
        for row in self.attendance_tree.get_children():
            self.attendance_tree.delete(row)
        
        self.cursor.execute('''
            SELECT a.date, e.name, a.status
            FROM MonthlyAttendance a
            JOIN Employee e ON a.employee_id = e.employee_id
        ''')
        for row in self.cursor.fetchall():
            self.attendance_tree.insert('', tk.END, values=row)

Part 9: Salary Tab Functionalities

In [13]:
    def create_date_range_selector(self, parent_frame):
        """Create date range selectors for the Salary Tab."""
        ttk.Label(parent_frame, text="From:").grid(row=0, column=0, padx=5, pady=5)
        self.salary_from_date = DateEntry(parent_frame, width=12)
        self.salary_from_date.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(parent_frame, text="To:").grid(row=0, column=2, padx=5, pady=5)
        self.salary_to_date = DateEntry(parent_frame, width=12)
        self.salary_to_date.grid(row=0, column=3, padx=5, pady=5)
    
    def calculate_salary(self):
        """Calculate salary for a selected employee within the given date range."""
        employee_name = self.salary_employee_var.get()
        from_date = self.salary_from_date.get_date()
        to_date = self.salary_to_date.get_date()
        
        if not employee_name:
            messagebox.showerror("Error", "Select an employee.")
            return
        
        # Get employee ID
        self.cursor.execute('SELECT employee_id, base_salary FROM Employee WHERE name = ?', (employee_name,))
        result = self.cursor.fetchone()
        if not result:
            messagebox.showerror("Error", "Employee not found.")
            return
        
        employee_id, base_salary = result
        try:
            # Fetch attendance data for the given period
            self.cursor.execute('''
                SELECT COUNT(*) 
                FROM MonthlyAttendance 
                WHERE employee_id = ? AND date BETWEEN ? AND ? AND status = 'Present'
            ''', (employee_id, from_date, to_date))
            present_days = self.cursor.fetchone()[0]
            
            salary = base_salary / 30 * present_days  # Example calculation
            self.salary_text.delete("1.0", tk.END)
            self.salary_text.insert(tk.END, f"Salary for {employee_name} ({from_date} to {to_date}):\n")
            self.salary_text.insert(tk.END, f"Base Salary: {base_salary}\n")
            self.salary_text.insert(tk.END, f"Present Days: {present_days}\n")
            self.salary_text.insert(tk.END, f"Calculated Salary: {salary}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to calculate salary. {e}")

Finalizing the App

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = EmployeeAttendanceApp(root)  # Ensure this method is called if needed
    root.mainloop()